In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import pickle

from BoundingBox import BoundingBox
from BoundingBoxes import BoundingBoxes
from Evaluator import *
from utils import *
from helper.nms import non_max_suppression_fast

In [6]:
path = Path('../../Slides/Equine/')

database = Database()
database.open(str(path/'EIPH.sqlite'))

True

In [7]:
files = []
lbl_bbox = []

getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    down_factor = 1

    classes = {3: 0, 4: 1, 5: 2, 6: 3, 7: 4}#{3: 1, 4: 2, 5: 3, 6: 4, 7: 5} #
    labels, bboxes = [], []
    for id, annotation in database.annotations.items():
        if annotation.labels[0].classId in classes:
            d = 2 * annotation.r / down_factor
            x_min = (annotation.x1 - annotation.r) / down_factor
            y_min = (annotation.y1 - annotation.r) / down_factor
            x_max = x_min + d
            y_max = y_min + d
            label = classes[annotation.labels[0].classId]

            bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
            labels.append(label)

    if len(bboxes) > 0:
        lbl_bbox.append([bboxes, labels])
        files.append(slide_path)

img2bbox = dict(zip(files, np.array(lbl_bbox)))
get_y_func = lambda o:img2bbox[o]

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:00<00:06,  3.36it/s]

Loading DB into memory ...


  8%|▊         | 2/24 [00:01<00:13,  1.59it/s]

...

Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:05<00:00,  4.74it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:06<00:00,  3.55it/s]

Loading DB into memory ...


In [8]:
inference_data = pickle.load(open("inference_results_boxes.p", "rb"))

data = {'File':[],
        'Th': [],
        'Th IOU': [],
        'Total GT':[],
        'Total Pred':[],
        'Mean GT':[],
        'Mean Pred':[],
        'Std GT':[],
        'Std Pred':[],
        'AP':[],
       }

In [9]:
for file in tqdm(files):
    boxes_gt, labels_gt = get_y_func(file)
    boxes_gt, labels_gt = np.array(boxes_gt), np.array(labels_gt)

    # change gt from x,y,x2,y2 -> x,y,w,h
    boxes_gt[:, 2:] = boxes_gt[:, 2:] - boxes_gt[:, :2]

    inference_data_image = np.array(inference_data[file.name])



    boundingBoxes = BoundingBoxes()
    evaluator = Evaluator()
    for th in range(30, 70, 5):
        for overlapThresh in range(10, 75, 5):

            score_pred = inference_data_image[:, 5]
            ids = score_pred > th / 100

            nms_result = non_max_suppression_fast(inference_data_image[ids], overlapThresh / 100)

            boxes_pred = nms_result[:, [0, 1, 2, 3]]
            # change pred from x,y,x2,y2 -> x,y,w,h
            boxes_pred[:, 2:] = boxes_pred[:, 2:] - boxes_pred[:, :2]
            labels_pred = nms_result[:, 4]
            score_pred = nms_result[:, 5]


            boundingBoxes.removeAllBoundingBoxes()

            ids = score_pred >= th

            for box, cla in zip(boxes_gt, labels_gt):
                temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                   y=box[1],
                                   w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                   bbType=BBType.GroundTruth, format=BBFormat.XYWH, imgSize=(5000, 5000))
                boundingBoxes.addBoundingBox(temp)

            for box, cla, scor in zip(boxes_pred, labels_pred, score_pred):
                temp = BoundingBox(imageName=str(file), classId=cla, x=box[0],
                                   y=box[1], classConfidence=scor,
                                   w=box[2], h=box[3], typeCoordinates=CoordinatesType.Absolute,
                                   bbType=BBType.Detected, format=BBFormat.XYWH, imgSize=(5000, 5000))
                boundingBoxes.addBoundingBox(temp)

            metricsPerClass = evaluator.GetPascalVOCMetrics(boundingBoxes, IOUThreshold=overlapThresh / 100)
            mean_ap = np.array([mc['AP'] for mc in metricsPerClass]).mean()

            data['File'].append(file.name)
            data['Th'].append(th / 100)
            data['Th IOU'].append(overlapThresh / 100)

            data['Total GT'].append(len(labels_pred))
            data['Total Pred'].append(len(labels_gt))

            data['Std GT'].append(labels_gt.std())
            data['Std Pred'].append(labels_pred.std())

            data['Mean GT'].append(labels_gt.mean())
            data['Mean Pred'].append(labels_pred.mean())

            data['AP'].append(mean_ap)

/home/c.marzahl@de.eu.local/ProgProjekte/Object-Detection-Metrics/lib/Evaluator.py:126: RuntimeWarning: invalid value encountered in true_divide
  rec = acc_TP / npos


In [10]:
data

{'File': ['01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '01_EIPH_563479 Berliner Blau.svs',
  '0